In [122]:
import re
import hashlib
from pycipher import SimpleSubstitution as SimpleSub
import random
from ngram_score import ngram_score
from operator import itemgetter

In [123]:
# from practicalcryptography.com
f = open("quadgrams.txt", "r")
qg = f.read()
# load our quadgram statistics
fitness = ngram_score(qg) 

In [124]:
ETAOIN  = 'etaoinshrdlcumwfgypbvkjxqz'
LETTERS = 'abcdefghijklmnopqrstuvwxyz'
cipher = ''
corpus = ''
h = ''
hc = ''
no_of_iterations = 10
maxkey=''

In [125]:
def slicer(text):
    return text[:]

def letter_count(text):
    
    #Returns a dictionary with the letter-frequency 
    #pair of the text parameter. 
    
    #Use a dictionary to store the the letter-frequency pair 
    letterCount = {}
    
    #Initialize the dictionary with 0 frequencies
    for letter in LETTERS:
        letterCount[letter] = 0   
        
    #Increment the occurrencies of the letters in the fext  
    for letter in text.lower():
        if letter in LETTERS:
            letterCount[letter] += 1
            
    return letterCount

def plot_letter_freq(letterCount):
   
    # Plot the histogram of the letter frequency pairs
    
    sorted_occ = sorted(letterCount.items(), key = itemgetter(1), reverse = True) 
    centers = range(len(LETTERS))
    plt.bar(centers, [i[1] for i in sorted_occ], align = 'center', tick_label = [i[0] for i in sorted_occ])
    plt.show()

def get_freq_order(message):

    #Returns a string of the alphabet letters arranged in order of most 
    #frequently occurring in the message parameter.
    
    #Dictionary of each letter and its frequency count (obtained with the previous function)
    letter2freq = letter_count(message)
     
    #Make a dictionary of each frequency count to each letter(s) with that frequency
    freq2letter = {}
    
    for key, value in letter2freq.items():
        if value in freq2letter:
            freq2letter[value].append(key)
        else:
            freq2letter[value] = [key]   
    
    #Sort the list of letters in reverse "ETAOIN" order and then convert them into a string
    #Use the "ETAOIN" letter frequency to determine the order of the letters with the same frequency 
    
    for freq in freq2letter: 
        #Sort the letters according to the ETAOIN.find output and in reverse order
        freq2letter[freq].sort(key = ETAOIN.find, reverse = False) 
        freq2letter[freq] = ''.join(freq2letter[freq])
    
    #Convert dictionaries with frequency-letter pairs into strings 
    
    freqPairs = list(freq2letter.items())   #Create a list of tuplets pairs (key, value)
    freqPairs.sort(key = itemgetter(0), reverse = True) #Sort them in ascending order according to value at index 0
    freqOrder = ''.join(list(map(itemgetter(1), freqPairs))) #map () maps all the element accessed using itemgetter() 
    
    return freqOrder

In [126]:
def manual_swap(cipher):
    key_swapped = 'epmuhxvsakyrfljdbqgntcziow'
    res = SimpleSub(key_swapped).decipher(cipher)
    return res

In [127]:
with open('ciphertext.txt', 'r') as c:
    cipher = c.read()
    cipher = slicer(cipher)

with open('corpus.txt', 'r') as c:
    corpus = c.read()
    corpus = slicer(corpus)

with open ('sha256sum.txt') as hf:
    h=hf.read()

In [128]:
sorted_freq_corpus = get_freq_order(corpus)
sorted_freq_cipher = get_freq_order(cipher)
# here wwe go for the manual swap to get the variable key_swapped used above
print('Freq Corpus : ', sorted_freq_corpus,'\nFreq Cipher : ', sorted_freq_cipher)

Freq Corpus :  etaonihsrdluwcmfgypbvkxjqz 
Freq Cipher :  hnejlasgqurtzmfvxodpcyikbw


In [121]:
ptext_cndt = manual_swap(cipher)
digest = hashlib.sha256(ptext_cndt.encode('utf-8')).hexdigest()

if digest == h:
    print('Found \nKey: ', sorted_freq_cipher)
else:
    cipher = cipher.lower()
    maxkey = list('abcdefghijklmnopqrstuvwxyz')
    maxscore = -99e9
    parentscore,parentkey = maxscore,maxkey[:]
    print ("\n..Brute Forcing the cypher..\nYou may have to wait several iterations")
    i = 0
    while (i<no_of_iterations and digest != h):
        random.shuffle(parentkey)
        deciphered = SimpleSub(parentkey).decipher(cipher, keep_punct=True).lower()
        parentscore = fitness.score(deciphered)
        count = 0
        while count < 1000:
            a = random.randint(0,25)
            b = random.randint(0,25)
            child = parentkey[:]
            # swap two characters in the child
            child[a],child[b] = child[b],child[a]
            deciphered = SimpleSub(child).decipher(cipher, keep_punct=True).lower()
            score = fitness.score(deciphered)
            # if the child was better, replace the parent with it
            if score > parentscore:
                parentscore = score
                parentkey = child[:]
                count = 0
            count = count+1
        # keep track of best score seen so far, keys with best scores are only used for decryption of the ciphertext now, each iteration and try is displayed
        if parentscore>maxscore:
            i = i+1
            maxscore,maxkey = parentscore,parentkey[:]
            print ('\nbest score so far:',maxscore)
            ss = SimpleSub(maxkey)
            plain_t = ss.decipher(cipher, keep_punct=True)
            print ('    best key: '+''.join(maxkey))
            print ('    plaintext: '+plain_t.lower())
            digest = hashlib.sha256(plain_t.lower().encode('utf-8')).hexdigest()
            print("The digest is : " , digest)
            if digest == h:
                print('Found!')

print('\n..\n..\nThe Correct key: '+''.join(maxkey))


..Brute Forcing the cypher..
You may have to wait several iterations

best score so far: -9310.422852695394
    best key: epfuhxvsakyrzljdwqgntcmiob
    plaintext: up at all!

the little princess got up, rang for the maid, and hurriedly and merrily
began to devise and carry out a plan of how princess mary should be
dressed. princess marys self-esteem was wounded by the fact that
the arrival of a suitor agitated her, and still more so by both
her companions not having the least conception that it could be
otherwise. to tell them that she felt ashamed for herself and for them
would be to betray her agitation, while to decline their offers to
dress her would prolong their banter and insistence. she flushed, her
beautiful eyes grew dim, red blotches came on her face, and it took
on the unattractive martyrlike expression it so often wore, as she
submitted herself to mademoiselle bourienne and lise. both these women
quite sincerely tried to make her look pretty. she was so plain that
neithe

KeyboardInterrupt: 